In [24]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import math

import os.path
import pickle
import tqdm

## Glicko Functions

In [20]:
def expected_placement_glicko(RATING, ALL_RATING, RD, ALL_RD, q):
  expected_placement = len(ALL_RATING) + 1

  for i in range(len(ALL_RATING)):
    expected_placement -= 1 / (1 + 10 ** ((-g(math.sqrt(RD**2 + ALL_RD[i]**2), q)*(RATING - ALL_RATING[i]))/400))

  return expected_placement - 0.5

def update_rating_rd_glicko(RATING, ALL_RATING, RD, ALL_RD, q, PLACEMENT):
    phi_sum = 0
    theta_sum = 0

    found_self = False
    for i in range(len(ALL_RATING)):
        if i + 1 != PLACEMENT:
            E = 1/(1 + 10**(-(g(math.sqrt(RD**2  + ALL_RD[i]**2), q)) * (RATING - ALL_RATING[i])/400))
            phi_sum += g(ALL_RD[i], q)*g(ALL_RD[i], q)*E*(1-E)
            theta_sum += g(ALL_RD[i], q)*((PLACEMENT <= i + 1) - E)
    d2 = 1/(q*q*phi_sum)
    return (RATING + theta_sum*q/(1/RD**2+1/d2), max(30,math.sqrt(1/(1/(RD**2)+1/d2))))

def g(RD, q):
    return 1/(math.sqrt(1 + (3*q*q*RD*RD)/(math.pi**2)))

# Import History and to Predict

In [32]:
to_predict = pd.read_excel('../data/to_predict.ods').T
rating_history = pd.read_parquet('../data/rating_history.parquet')
rd_history = pd.read_parquet('../data/rd_history.parquet')
predicted_events = pd.DataFrame(columns = ['event', 'date', 'player_count', 'players'])
for index, row in to_predict.iterrows():
    if index != 'Event':
        event = index
        date = row.iloc[0]
        player_count = row.iloc[1]
        players = list(row.iloc[2:].dropna())
        predicted_events.loc[len(predicted_events)] = (pd.Series({'event' : event, 'date' : date, 'player_count' : player_count, 'players' : players}))

predictions=dict()

# Calculate Predictions

In [40]:
q = math.log(10)/400
c = 15
d = 20
starting_rating = 1000.0
starting_rd = 200.0


for i in range(len(predicted_events)):
    event_name = predicted_events.at[i, "event"]
    this_event = dict(predicted_events.iloc[i])
    this_event_preds = pd.DataFrame(columns = np.array(["Player", "Rating", "Global", "EP"]))
    this_event_stats = {'NP EP' : 0, 'NP RC First' : 0, 'NP RC Last' : 0}

    playing = []
    prior_rating = {}
    prior_rd = {}
    expected = {}

    # Register current players

    for player in this_event['players']:
        player_id = player.lower().replace("_", "")
        playing.append(player_id)

    for player in this_event['players']:
        player_id = player.lower().replace("_", "")

        # Register prior stats

        if player_id not in rankings.index:
            prior_rating[player_id] = starting_rating
            prior_rd[player_id] = starting_rd
        else:
            prior_rating[player_id] = rating_history.at[player_id, "Rating"]
            prior_rd[player_id] = rd_history.at[player_id, "RD"]

    prior_rankings = rating_history.sort_values(by = "Rating", ascending = False)

    for player in this_event['players']:
        player_id = player.lower().replace("_", "")

        # Calculate Expected

        expected[player_id] = expected_placement_glicko(
            prior_rating[player_id], 
            list(prior_rating.values()), 
            prior_rd[player_id], 
            list(prior_rd.values()), 
            q
        )

        # Calculate and Update Rating and RD
        '''
        new_rating[player_id], new_rd[player_id] = update_rating_rd_glicko(
            prior_rating[player_id], 
            list(prior_rating.values()), 
            prior_rd[player_id], 
            list(prior_rd.values()), 
            q, 
            actual[player_id]
        )

        rating_history.at[player_id, "Rating"] = new_rating[player_id]
        rd_history.at[player_id, "RD"] = new_rd[player_id]
        rating_history.at[player_id, event_name] = new_rating[player_id]
        rd_history.at[player_id, event_name] = new_rd[player_id]
        rating_history.at[player_id, "Events"] += 1
        rating_history.at[player_id, "I"] = 0
        '''

    for player in this_event['players']:
        player_id = player.lower().replace("_", "")

        try: 
            this_event_preds.loc[player_id] = {
                "Player" : player,
                "Rating" : prior_rating[player_id],
                "Global" : prior_rankings.index.get_loc(player_id) + 1,
                "EP" : expected[player_id],
            }
        except:
                this_event_preds.loc[player_id] = {
                "Player" : player,
                "Rating" : prior_rating[player_id],
                "Global" : "?",
                "EP" : expected[player_id],
            }
        
    this_event['predictions'] = this_event_preds

    # Event stats

    this_event_stats['NP EP'] = expected_placement_glicko(
            starting_rating, 
            list(prior_rating.values()), 
            starting_rd, 
            list(prior_rd.values()), 
            q
        )

    temp = 0
    this_event_stats['NP RC First'], temp = update_rating_rd_glicko(
            starting_rating, 
            list(prior_rating.values()), 
            starting_rd, 
            list(prior_rd.values()), 
            q, 
            1
        )

    this_event_stats['NP RC Last'], temp = update_rating_rd_glicko(
            starting_rating, 
            list(prior_rating.values()), 
            starting_rd, 
            list(prior_rd.values()), 
            q, 
            len(prior_rating.values())
        )

    this_event['stats'] = this_event_stats

    predictions[event_name] = this_event

In [41]:
predictions['PBTLD']['predictions']

,Player,Rating,Global,EP
soultb,SoulTB,1805.995298,6,9.083175
phazeray,PhazeRay,1407.369437,82,20.344729
brinkz,_Brinkz,1650.795637,19,13.367329
jackcas,JackCas,1269.896651,159,23.822010
firebreathman,FireBreathMan,1781.236695,7,9.730718
ubhi,ubhi,1727.447092,11,11.164139
c10wd,C10WD,1410.048153,81,20.258197
unixn,unixn,1341.039618,113,22.014192
sandwichlord,SandwichLord_,1813.863947,5,8.869019
cgq,CGQ_,1592.305169,26,15.066044


In [42]:
with open('../data/predictions.pkl', 'wb') as file:
    pickle.dump(predictions, file)

